# Issue with Branch cut in Bragg exact

For what should be the guided mode region, we have a branch cut at the real line currently.  StepExact doesn't have it, and it's derived from Marcuse (Light Transmission Optics).  Here we try to figure out why this is happening.

### Hand implement Marcuse Matrix

Marcuse simplifies the determinant.  Maybe the issue is in taking determinant from the matrix directly.

In [ ]:
import numpy as np
from scipy.special import jv, jvp, h1vp, hankel1
from step_exact import plotlogf
from fiberamp.fiber.microstruct.bragg import BraggExact

We have an implementation to give us relevant parameter values from the Nufern fiber, so we'll use it

In [ ]:
n_core = 1.450971
n_clad = 1.4497299206545333

A = BraggExact(ts=[1.25e-5, 2e-5], mats=['core', 'clad'], 
               ns=[lambda x:n_core, lambda x: n_clad],
               maxhs=[.1,.1], wl=1.064e-6, scale=1.25e-5)


In [ ]:
k_low = A.k0 * A.ns[1] * A.scale
k_high = A.k0 * A.ns[0] * A.scale
outer = 'h2'
nu = 1
k_low, k_high

In [ ]:
%matplotlib inline
plotlogf(A.determinant, k_low-.1, k_high+.01, -.03, .03, nu, outer,
         iref=100, rref=100, levels=100, figsize=(10,7))

The branch cut looks like this:  (hopefully it won't soon)

In [ ]:
from IPython.display import Image
Image("branchcut.jpg")

Issue appears to have been with definition of Kappa: Kappa^2 = k_i^2 - beta^2.  Marcuse uses i * Gamma where Gamma^2 = beta^2 - k_i^2.  When using the previous definition in the outer region with guided modes we get the branch cut.  When using Marcuse's Gamma the branch cut goes away.  Changed implementation of BraggExact to use Marcuse's Gamma when finding guided modes.  When finding leaky modes (H2 in outer region), we appear to have a branch cut 